In [1]:
import duckdb

In [2]:
con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL ducklake; LOAD ducklake;")

In [3]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'localhost:9000'
}

con.sql(f"SET s3_endpoint = '{minioAuth['endpoint']}';") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_use_ssl = false;") # Wajib untuk http (tanpa ssl)
con.sql("SET s3_url_style = 'path';") # Wajib untuk MinIO

In [5]:
ducklakeCatalog = '../catalog.ducklake'
ducklakeDataPath = f"s3://{minioAuth['bucket']}/data-lakehouse/managedStoreData/"

In [6]:
con.sql(f"ATTACH 'ducklake:{ducklakeCatalog}' AS dataLakehouse (DATA_PATH '{ducklakeDataPath}');")
con.sql("USE dataLakehouse;")

In [7]:
con.sql("SELECT * FROM ducklake_snapshots('dataLakehouse')")

┌─────────────┬───────────────────────────────┬────────────────┬─────────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                               changes                               │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                       map(varchar, varchar[])                       │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼─────────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2025-12-08 17:43:01.726776+07 │              0 │ {schemas_created=[main]}                                            │ NULL    │ NULL           │ NULL              │
│           1 │ 2025-12-08 17:43:39.987409+07 │              1 │ {schemas_created=[staging

In [9]:
con.close()